In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import scipy.io
import math

# format: virtualflies x (PPL1_01, 02, .., 05, odor)
# arranged odor #0 - #4
virtualFly = scipy.io.loadmat('virtualFly_PPL1.mat')['virtualFly']

In [3]:
ODOR_NUM = 5
PPL1_NUM = 5
NUM_FLIES_PER_ODOR = 12

virtualFly.shape

VF_OD1 = virtualFly[:NUM_FLIES_PER_ODOR**PPL1_NUM,:]
VF_OD2 = virtualFly[NUM_FLIES_PER_ODOR**PPL1_NUM:2*NUM_FLIES_PER_ODOR**PPL1_NUM,:]
VF_OD3 = virtualFly[2*NUM_FLIES_PER_ODOR**PPL1_NUM:3*NUM_FLIES_PER_ODOR**PPL1_NUM,:]
VF_OD4 = virtualFly[3*NUM_FLIES_PER_ODOR**PPL1_NUM:4*NUM_FLIES_PER_ODOR**PPL1_NUM,:]
VF_OD5 = virtualFly[4*NUM_FLIES_PER_ODOR**PPL1_NUM:5*NUM_FLIES_PER_ODOR**PPL1_NUM,:]

'''
print(VF_OD1.shape)
print(VF_OD2.shape)
print(VF_OD3.shape)
print(VF_OD4.shape)
print(VF_OD5.shape)
'''

#shuffle data within the same odor
np.random.shuffle(VF_OD1)
np.random.shuffle(VF_OD2)
np.random.shuffle(VF_OD3)
np.random.shuffle(VF_OD4)
np.random.shuffle(VF_OD5)



In [4]:
# dataset arrange

numTrain = virtualFly.shape[0] * 0.9
numVal = virtualFly.shape[0] * 0.05
numTest = virtualFly.shape[0] * 0.05
numTrainPerOD = math.floor(numTrain/ODOR_NUM)
numValPerOD = math.floor(numVal/ODOR_NUM)
numTestPerOD = math.floor(numTest/ODOR_NUM)
'''
print(numTrain - numTrainPerOD*ODOR_NUM)
print(numVal - numValPerOD*ODOR_NUM)
print(numTest - numTestPerOD*ODOR_NUM)
print(virtualFly.shape[0]/5 - numTrainPerOD - numValPerOD - numTestPerOD)
print(numTrainPerOD, numValPerOD, numTestPerOD)
'''
mask = range(numTrainPerOD)
trainSet = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 
np.random.shuffle(trainSet)
X_train = trainSet[:,:PPL1_NUM]
y_train = trainSet[:,-1]-1 # -1 to conform python index from 0 to 4 (class imported from MATLAB was from 1 to 5)
X_train_shuffle = np.copy(X_train)
np.random.shuffle(X_train_shuffle)

mask = range(numTrainPerOD, numTrainPerOD+numValPerOD)
valSet = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 
np.random.shuffle(valSet)
X_val = valSet[:,:PPL1_NUM]
y_val = valSet[:,-1]-1 # to conform python index format

mask = range(numTrainPerOD+numValPerOD, numTrainPerOD+numValPerOD+numTestPerOD)
testSet = np.concatenate((VF_OD1[mask,:], VF_OD2[mask,:], VF_OD3[mask,:], VF_OD4[mask,:], VF_OD5[mask,:]),axis=0) 
np.random.shuffle(testSet)
X_test = testSet[:,:PPL1_NUM]
y_test = testSet[:,-1]-1 # to conform python index format

print(numTrain, trainSet.shape)
print(numVal, valSet.shape)
print(numTest, testSet.shape)

print('Train shape: ', X_train.shape)
print('Validation shape: ', X_val.shape)
print('Test shape: ', X_test.shape)


1119744.0 (1119740, 6)
62208.0 (62205, 6)
62208.0 (62205, 6)
Train shape:  (1119740, 5)
Validation shape:  (62205, 5)
Test shape:  (62205, 5)


In [5]:
# preprocessing

#-mean - should do std as well?
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
print(mean,'\n', std)

X_train -= mean
X_train_shuffle -= mean
X_val -= mean
X_test -= mean

'''
X_train /= std
X_val /= std
X_test /= std
'''

[ 1.21204502  7.02026588 -0.05224128  2.780903    8.15481371] 
 [6.55257196 6.68008451 5.11382304 6.89063755 3.84040533]


'\nX_train /= std\nX_val /= std\nX_test /= std\n'

In [17]:
def train(X, Y, XV, YV, model, lr, reg, batchSize, epoch):
    X = torch.Tensor(X)
    Y = torch.Tensor(Y)
    #X = torch.FloatTensor(X)
    #Y = torch.FloatTensor(Y)
    N = len(Y)
    XV = torch.FloatTensor(XV)
    YV = torch.FloatTensor(YV)

    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    optimizer = optim.Adam(model.parameters(), lr=lr)


   
    for epoch in range(epoch):
        perm = torch.randperm(N)
        sum_loss = 0
        model.train()
        for i in range(0, N, batchSize):
            x = X[perm[i:i+batchSize]]
            y = Y[perm[i:i+batchSize]]

            optimizer.zero_grad()
            weight = model.weight
            bias = model.bias
            output = model(x)
            
            #print(weight)
            #print(bias)
            #print(x)
            #print(x.double() @ weight + bias.double())
            #print(torch.mm(x, torch.t(model.weight))
            #print(torch.addmm(bias, x, torch.t(weight)))
            
            '''
            print("x:",x)
            print("y:",y.long())
            print("output:",output)
            '''
            
            correct_class_score = output[torch.arange(x.shape[0]), y.long()]
            
            margin = torch.maximum(torch.zeros(output.shape), output - correct_class_score.unsqueeze(1) + 1) # delta is 1
            loss = (torch.sum(margin)-1*x.shape[0])/x.shape[0] # remove correct case when delta is 1
            #print("loss1:", loss)
            loss += reg * torch.sum((weight.t() @ weight)) #/ 2.0
            #print("loss2:", loss)
            
            '''
            print(output - correct_class_score.unsqueeze(1)+1)
            print(torch.zeros(output.shape))
            print(correct_class_score)
            print("loss:", loss)
            '''
            #loss = torch.mean(torch.clamp(1 - y * output, min=0))
            
            loss.backward()
            optimizer.step()

            sum_loss += float(loss)
            

        print("Epoch: {:4d}\tloss: {}".format(epoch, sum_loss / N))

        numScore = 0
        model.eval()
        for i in range(0, len(YV), batchSize):
            x = XV[i:i+batchSize]
            y = YV[i:i+batchSize]
            
            optimizer.zero_grad()
            with torch.no_grad():
                output = model(x).numpy()
                pred = np.argmax(output, axis = 1)
                #print(pred)
                #print(y.numpy())
                numScore += np.sum((pred == y.numpy()))
                #print(numScore)
            
        
        print("validation acuuracy: {}".format(numScore / len(YV)))
        
def test(X, Y, model, batchSize, numOnly = False):
    XT = torch.FloatTensor(X)
    YT = torch.FloatTensor(Y)
    numScore = 0
    model.eval()
    for i in range(0, len(YT), batchSize):
        x = XT[i:i+batchSize]
        y = YT[i:i+batchSize]
        output = model(x).detach().numpy()
        pred = np.argmax(output, axis = 1)
        #print(pred)
        #print(y.numpy())
        numScore += np.sum((pred == y.numpy()))
        #print(numScore)
    if numOnly:
        print(numScore / len(YT))
    else:
        print("test acuuracy: {}".format(numScore / len(YT)))


        
def visualize(X, Y, model):
    W = model.weight.squeeze().detach().cpu().numpy()
    b = model.bias.squeeze().detach().cpu().numpy()

    delta = 0.001
    x = np.arange(X[:, 0].min(), X[:, 0].max(), delta)
    y = np.arange(X[:, 1].min(), X[:, 1].max(), delta)
    x, y = np.meshgrid(x, y)
    xy = list(map(np.ravel, [x, y]))

    z = (W.dot(xy) + b).reshape(x.shape)
    z[np.where(z > 1.0)] = 4
    z[np.where((z > 0.0) & (z <= 1.0))] = 3
    z[np.where((z > -1.0) & (z <= 0.0))] = 2
    z[np.where(z <= -1.0)] = 1

    plt.figure(figsize=(10, 10))
    plt.xlim([X[:, 0].min() + delta, X[:, 0].max() - delta])
    plt.ylim([X[:, 1].min() + delta, X[:, 1].max() - delta])
    plt.contourf(x, y, z, alpha=0.8, cmap="Greys")
    plt.scatter(x=X[:, 0], y=X[:, 1], c="black", s=10)
    plt.tight_layout()
    plt.show()


In [458]:
''' cur best ~77%
SGD
lr = 0.05
reg = 0.001
Adam 77.5%
lr = 0.001
reg = 0.001
'''
lr = 0.0005
reg = 0.0001

batchSize = 200
epoch = 10
    
model = nn.Linear(5, 5)
print (list(model.parameters()))
#print(model.weight)
#print(model.bias)
#print(torch.nn.Parameter.data)

#print(xx.dtype)
#print(model(xx) - torch.matmul(xx,torch.t(model.weight)))

train(X_train, y_train, X_val, y_val, model, lr, reg, batchSize, epoch)
test(X_test, y_test, model, batchSize)


print("Shuffle Begins")
model_shuffle = nn.Linear(5,5);
train(X_train_shuffle, y_train, X_val, y_val, model_shuffle, lr, reg, batchSize, epoch)
test(X_test, y_test, model_shuffle, batchSize)



[Parameter containing:
tensor([[-0.2470,  0.0653, -0.3567,  0.3030,  0.0298],
        [ 0.2051,  0.1397, -0.4210, -0.0158, -0.2440],
        [ 0.0085, -0.1184,  0.4108,  0.3798,  0.1309],
        [-0.0208, -0.0765, -0.1624, -0.0473,  0.3400],
        [-0.2556, -0.2783,  0.3246, -0.1986,  0.0520]], requires_grad=True), Parameter containing:
tensor([-0.1394,  0.1393, -0.3365, -0.3078, -0.2304], requires_grad=True)]
Epoch:    0	loss: 0.006064423562796383
validation acuuracy: 0.7715456956836267
Epoch:    1	loss: 0.0030992623335462993
validation acuuracy: 0.7741178361868017
Epoch:    2	loss: 0.0030032954377750497
validation acuuracy: 0.7740053050397878
Epoch:    3	loss: 0.0029852469740785305
validation acuuracy: 0.7740535326742223
Epoch:    4	loss: 0.002980184014042577
validation acuuracy: 0.7745518848967125
Epoch:    5	loss: 0.0029782942128864513
validation acuuracy: 0.7741499879430914
Epoch:    6	loss: 0.0029777735598414744
validation acuuracy: 0.7744232778715537
Epoch:    7	loss: 0.00297

In [313]:
print("Shuffle Begins")
model_shuffle = nn.Linear(6,5);
train(X_train_shuffle, y_train, X_val, y_val, model_shuffle, lr, reg, batchSize, epoch)
test(X_test, y_test, model_shuffle, batchSize)

TODO
1. merge val+test sets and divided to trials with 1000 data per each trial run test per each and gather statistics
    a. odor distribution ratio should be the same to make the chance level all the same
2. do the same analysis for MBONs
3. do the same analysis for PPL1+MBONs

In [7]:
lr = 0.0005
reg = 0.0001

batchSize = 200
epoch = 10
    
model = nn.Linear(5, 5)
print (list(model.parameters()))
#print(model.weight)
#print(model.bias)
#print(torch.nn.Parameter.data)

#print(xx.dtype)
#print(model(xx) - torch.matmul(xx,torch.t(model.weight)))

train(X_train, y_train, X_val, y_val, model, lr, reg, batchSize, epoch)
test(X_test, y_test, model, batchSize)

[Parameter containing:
tensor([[ 0.2794,  0.2948, -0.2136,  0.4029,  0.0353],
        [-0.3357,  0.1529,  0.1358,  0.1007, -0.0047],
        [ 0.1396, -0.0767,  0.1203,  0.3329, -0.3400],
        [-0.0840, -0.2249,  0.3774,  0.2884, -0.0547],
        [-0.3600,  0.4233, -0.0028, -0.1043, -0.3285]], requires_grad=True), Parameter containing:
tensor([-0.1602, -0.1086, -0.0725, -0.2676, -0.4049], requires_grad=True)]
Epoch:    0	loss: 0.007196016061844056
validation acuuracy: 0.7738445462583393
Epoch:    1	loss: 0.0031107783623376805
validation acuuracy: 0.7748894783377542
Epoch:    2	loss: 0.003005908718612609
validation acuuracy: 0.7752431476569408
Epoch:    3	loss: 0.002986332149365895
validation acuuracy: 0.7754842858291134
Epoch:    4	loss: 0.0029806031067059605
validation acuuracy: 0.7754039064383892
Epoch:    5	loss: 0.0029787902851503073
validation acuuracy: 0.7756128928542722
Epoch:    6	loss: 0.002977642000636032
validation acuuracy: 0.7756128928542722
Epoch:    7	loss: 0.0029775

In [8]:
print("Shuffle Begins")
model_shuffle = nn.Linear(5,5);
train(X_train_shuffle, y_train, X_val, y_val, model_shuffle, lr, reg, batchSize, epoch)
test(X_test, y_test, model_shuffle, batchSize)

Shuffle Begins
Epoch:    0	loss: 0.020627771494977822
validation acuuracy: 0.17951933124346917
Epoch:    1	loss: 0.020000351809274876
validation acuuracy: 0.35858853789888273
Epoch:    2	loss: 0.019997139671563545
validation acuuracy: 0.3047986496262358
Epoch:    3	loss: 0.019997789764084062
validation acuuracy: 0.18377943895185275
Epoch:    4	loss: 0.019995576392278405
validation acuuracy: 0.2868579696165903
Epoch:    5	loss: 0.01999649148534408
validation acuuracy: 0.24446587894863758
Epoch:    6	loss: 0.019996979661512242
validation acuuracy: 0.22649304718270236
Epoch:    7	loss: 0.019995595183811887
validation acuuracy: 0.1746644160437264
Epoch:    8	loss: 0.019997563042651673
validation acuuracy: 0.28288722771481395
Epoch:    9	loss: 0.019995755795371117
validation acuuracy: 0.2039707419017764
test acuuracy: 0.20262036813760953


In [13]:
print(valSet.shape)
print(testSet.shape)
fullTestSet = np.concatenate((valSet,testSet),axis=0)
print(fullTestSet.shape)
np.random.shuffle(fullTestSet)
X_fullTest = fullTestSet[:,:PPL1_NUM]
y_fullTest = fullTestSet[:,-1]-1 # to conform python index format
print(X_fullTest.shape)
print(y_fullTest.shape)

(62205, 6)
(62205, 6)
(124410, 6)
(124410, 5)
(124410,)


In [19]:
sampleSize = 1000

#test for 120 cases
for i in range (0,120,1):
    test(X_fullTest[i*sampleSize:(i+1)*sampleSize,:], y_fullTest[i*sampleSize:(i+1)*sampleSize], model, batchSize, True)

0.778
0.788
0.779
0.77
0.775
0.77
0.765
0.793
0.787
0.749
0.766
0.774
0.782
0.776
0.774
0.793
0.785
0.76
0.77
0.778
0.777
0.781
0.769
0.773
0.768
0.776
0.793
0.791
0.762
0.751
0.774
0.778
0.786
0.772
0.793
0.791
0.785
0.773
0.776
0.785
0.757
0.786
0.774
0.757
0.783
0.775
0.762
0.783
0.772
0.809
0.789
0.761
0.777
0.801
0.749
0.789
0.772
0.78
0.762
0.76
0.796
0.784
0.749
0.782
0.767
0.76
0.77
0.763
0.76
0.801
0.784
0.76
0.777
0.779
0.785
0.766
0.808
0.774
0.769
0.767
0.779
0.783
0.768
0.789
0.768
0.766
0.793
0.774
0.784
0.776
0.778
0.786
0.773
0.766
0.787
0.771
0.783
0.777
0.773
0.784
0.767
0.782
0.774
0.771
0.764
0.798
0.773
0.77
0.766
0.76
0.781
0.786
0.757
0.764
0.763
0.763
0.763
0.776
0.763
0.774


In [18]:
sampleSize = 1000

#test for 120 cases
for i in range (0,120,1):
    test(X_fullTest[i*sampleSize:(i+1)*sampleSize,:], y_fullTest[i*sampleSize:(i+1)*sampleSize], model_shuffle, batchSize, True)

0.223
0.195
0.207
0.194
0.197
0.207
0.213
0.206
0.21
0.212
0.192
0.217
0.193
0.192
0.192
0.216
0.196
0.211
0.181
0.21
0.234
0.19
0.19
0.185
0.204
0.218
0.191
0.182
0.186
0.196
0.192
0.209
0.198
0.219
0.218
0.205
0.201
0.209
0.21
0.208
0.199
0.189
0.204
0.198
0.207
0.196
0.231
0.214
0.205
0.188
0.187
0.196
0.209
0.211
0.186
0.195
0.212
0.239
0.2
0.194
0.207
0.195
0.215
0.196
0.196
0.22
0.218
0.194
0.221
0.197
0.217
0.21
0.216
0.22
0.174
0.218
0.202
0.205
0.213
0.175
0.234
0.226
0.229
0.216
0.184
0.217
0.194
0.193
0.196
0.215
0.21
0.218
0.222
0.202
0.206
0.192
0.176
0.198
0.193
0.201
0.199
0.2
0.188
0.206
0.177
0.187
0.196
0.184
0.177
0.182
0.208
0.227
0.197
0.211
0.215
0.208
0.194
0.197
0.207
0.202


In [438]:
a = torch.tensor([1,2,3,4,5])
b = a.unsqueeze(1)
c = np.copy(a)
c -= 1
print(a)
print(b)
print(c)

tensor([1, 2, 3, 4, 5])
tensor([[1],
        [2],
        [3],
        [4],
        [5]])
[0 1 2 3 4]


In [1]:
print(numTrainPerOD)
print(numValPerOD)
print(numTestPerOD)

NameError: name 'numTrainPerOD' is not defined